In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns=['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns=['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('data/boston.csv')
data.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
### Ваш код ###
from sklearn.model_selection import train_test_split

columns = list(data.columns)
columns.remove('MEDV')

X = data[columns]
y = data['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=RANDOM_STATE)

display((X_train.shape, X_test.shape, y_train.shape, y_test.shape))

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
### Ваш код ###
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

linear = LinearRegression()
linear.fit(X_train, y_train)

ridge = Ridge()
ridge.fit(X_train, y_train)

lasso = Lasso()
lasso.fit(X_train, y_train)

# R2
r2_lr = r2_score(y_test, linear.predict(X_test))
r2_ridge = r2_score(y_test, ridge.predict(X_test))
r2_lasso = r2_score(y_test, lasso.predict(X_test))

results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

display(results_regression)

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [6]:
### Ваш код ###
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV

alphas = [10**i for i in range(-5, 6)]
parameters = {
    'alpha': alphas
}

ridge_gscv = GridSearchCV(Ridge(), parameters)
ridge_gscv.fit(X_train, y_train)

lasso_gscv = GridSearchCV(Lasso(), parameters)
lasso_gscv.fit(X_train, y_train)

ridge_cv = RidgeCV(alphas=alphas)
ridge_cv.fit(X_train, y_train)

lasso_cv = LassoCV(alphas=alphas)
lasso_cv.fit(X_train, y_train)

# R2
r2_ridge_grid_search = r2_score(y_test, ridge_gscv.predict(X_test))
r2_ridge_cv = r2_score(y_test, ridge_cv.predict(X_test))
r2_lasso_grid_search = r2_score(y_test, lasso_gscv.predict(X_test))
r2_lasso_cv = r2_score(y_test, lasso_cv.predict(X_test))

results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

display(results_regression)

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483


**Вывод:**

Модели Ridge и Lasso с подобранным коэффициентом регуляризации лучше моделей с коэффициентом регуляризации указанным по умолчанию, но разница не большая.

4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [7]:
### Ваш код ###
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

ridge_ss_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])
ridge_ss_pipe.fit(X_train, y_train)

ridge_mm_pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('model', Ridge())
])
ridge_mm_pipe.fit(X_train, y_train)

lasso_ss_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', Lasso())
])
lasso_ss_pipe.fit(X_train, y_train)

lasso_mm_pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('model', Lasso())
])
lasso_mm_pipe.fit(X_train, y_train)

# R2
r2_ridge_standart_scaler = r2_score(y_test, ridge_ss_pipe.predict(X_test))
r2_ridge_min_max_scaler = r2_score(y_test, ridge_mm_pipe.predict(X_test))
r2_lasso_standart_scaler = r2_score(y_test, lasso_ss_pipe.predict(X_test))
r2_lasso_min_max_scaler = r2_score(y_test, lasso_mm_pipe.predict(X_test))

results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

display(results_regression[3:11])

,model,task,R2
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045
10,Lasso_MinMaxScaler,task4,0.257392


**Вывод:**

- Для модели Ridge масштабирование MinMaxScaler немного улучшает модель (масштабирование StandardScaler дает примерной одниаковый результат).
- Для модели Lasso масштабирование MinMaxScaler и StandardScaler ухудшило качество модели особенно масштабирование MinMaxScaler.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [8]:
### Ваш код ###
alphas = [10**i for i in range(-5, 6)]

ridge_ss_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', RidgeCV(alphas=alphas)),
])
ridge_mm_pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('model', RidgeCV(alphas=alphas)),
])
lasso_ss_pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('model', LassoCV(alphas=alphas))
])
lasso_mm_pipe = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('model', LassoCV(alphas=alphas))
])

# R2
r2_ridge_standart_scaler_cv = r2_score(y_test, ridge_ss_pipe.fit(X_train, y_train).predict(X_test))
r2_ridge_min_max_scaler_cv = r2_score(y_test, ridge_mm_pipe.fit(X_train, y_train).predict(X_test))
r2_lasso_standart_scaler_cv = r2_score(y_test, lasso_ss_pipe.fit(X_train, y_train).predict(X_test))
r2_lasso_min_max_scaler_cv = r2_score(y_test, lasso_mm_pipe.fit(X_train, y_train).predict(X_test))

results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

display(results_regression[7:15])

,model,task,R2
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045
10,Lasso_MinMaxScaler,task4,0.257392
11,Ridge_StandardScaler_CV,task5,0.665731
12,Ridge_MinMaxScaler_CV,task5,0.669765
13,Lasso_StandardScaler_CV,task5,0.668482
14,Lasso_MinMaxScaler_CV,task5,0.668484


**Вывод:**

- Для модели Ridge масштабирование и подбор коэффициента регуляризации практически не улучшил качество модели.
- Для модели Lasso масштабирование и подбор коэффициента регуляризации улучшает модель по сравнению с предыдущим результатом (особенно для варианта, где используется масштабирование MinMaxScaler).

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [9]:
### Ваш код ###
from sklearn.preprocessing import PolynomialFeatures

ridge_ss_poly = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', Ridge()),
])
ridge_mm_poly = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', Ridge()),
])
lasso_ss_poly = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', Lasso()),
])
lasso_mm_poly = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', Lasso()),
])

# R2
r2_ridge_standart_scaler_poly = r2_score(y_test, ridge_ss_poly.fit(X_train, y_train).predict(X_test))
r2_ridge_min_max_scaler_poly = r2_score(y_test, lasso_ss_poly.fit(X_train, y_train).predict(X_test))
r2_lasso_standart_scaler_poly = r2_score(y_test, ridge_mm_poly.fit(X_train, y_train).predict(X_test))
r2_lasso_min_max_scaler_poly = r2_score(y_test, lasso_mm_poly.fit(X_train, y_train).predict(X_test))

results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

display(results_regression[11:19])

,model,task,R2
11,Ridge_StandardScaler_CV,task5,0.665731
12,Ridge_MinMaxScaler_CV,task5,0.669765
13,Lasso_StandardScaler_CV,task5,0.668482
14,Lasso_MinMaxScaler_CV,task5,0.668484
15,Ridge_StandardScaler_Poly,task6,0.817136
16,Ridge_MinMaxScaler_Poly,task6,0.732274
17,Lasso_StandardScaler_Poly,task6,0.829886
18,Lasso_MinMaxScaler_Poly,task6,0.261126


**Вывод:**

Попарные произведения признаков и их квадраты плюс масштабирование StandardScaler существенно улучшают качество моделей Ridge и Lasso. Масштабирование MinMaxScaler дает улучшение только для модели Ridge, для модели Lasso результат очень плохой.

7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [10]:
### Ваш код ###
ridge_ss_poly = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', RidgeCV(alphas=alphas)),
])
ridge_mm_poly = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', RidgeCV(alphas=alphas)),
])
lasso_ss_poly = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', LassoCV(alphas=alphas)),
])
lasso_mm_poly = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('preprocessor', PolynomialFeatures(degree=2, include_bias=False)),
    ('model', LassoCV(alphas=alphas)),
])

# R2
r2_ridge_standart_scaler_poly_cv = r2_score(y_test, ridge_ss_poly.fit(X_train, y_train).predict(X_test))
r2_ridge_min_max_scaler_poly_cv = r2_score(y_test, ridge_mm_poly.fit(X_train, y_train).predict(X_test))
r2_lasso_standart_scaler_poly_cv = r2_score(y_test, lasso_ss_poly.fit(X_train, y_train).predict(X_test))
r2_lasso_min_max_scaler_poly_cv = r2_score(y_test, lasso_mm_poly.fit(X_train, y_train).predict(X_test))

results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

display(results_regression[15:23])

,model,task,R2
15,Ridge_StandardScaler_Poly,task6,0.817136
16,Ridge_MinMaxScaler_Poly,task6,0.732274
17,Lasso_StandardScaler_Poly,task6,0.829886
18,Lasso_MinMaxScaler_Poly,task6,0.261126
19,Ridge_StandardScaler_Poly_CV,task7,0.818734
20,Ridge_MinMaxScaler_Poly_CV,task7,0.850140
21,Lasso_StandardScaler_Poly_CV,task7,0.812814
22,Lasso_MinMaxScaler_Poly_CV,task7,0.839702


**Вывод:**

В целом для моделей Ridge и Lasso результат получился самый лучший (для модели Lasso масштабирование StandardScaler немного ухудшилось качество по сравнению с предыдущим результатом).

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [11]:
### Ваш код ###
parameters = {
    'scaler': [StandardScaler(), MinMaxScaler()],
    'preprocessor__degree': list(range(1, 4)),
    'model': [Ridge(), Lasso()],
    'model__alpha': [10**i for i in range(-5, 6)],
}
pipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('preprocessor', PolynomialFeatures(include_bias=False)),
    ('model', Ridge()),
])
gs = GridSearchCV(estimator=pipe, param_grid=parameters)
gs.fit(X_train, y_train)

best_params = gs.best_params_
print('Параметры лучшей модели:\n', best_params)
r2_best_model = r2_score(y_test, gs.predict(X_test))
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

display(results_regression[23:])

Параметры лучшей модели:
 {'model': Lasso(alpha=0.001), 'model__alpha': 0.001, 'preprocessor__degree': 3, 'scaler': MinMaxScaler()}


,model,task,R2
23,Best_Model,task8,0.845838


**Вывод:**

- Наилучшая модель **Lasso**
- Наилучший коэффициент регуляризации **0.001**
- Наилучший метод масштабирования **MinMaxScaler**
- Наилучшая степень полинома **3**
- R2 для модели с текущими параметрами **0.845838**

In [12]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668475
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
7,Ridge_StandardScaler,task4,0.668190
8,Ridge_MinMaxScaler,task4,0.676221
9,Lasso_StandardScaler,task4,0.624045


http://archive.ics.uci.edu/ml/datasets/Adult

In [13]:
data = pd.read_csv('data/adult.csv')
data.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [14]:
### Ваш код ###
data['class'].replace('<=50K', 1, inplace=True)
data['class'].replace('>50K', 0, inplace=True)
data['class'].astype(int)

columns = list(data.columns)
columns.remove('class')

X = data[columns]
y = data['class']

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [15]:
### Ваш код ###
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

y_true = pd.Series([1] * y.shape[0])

f1_most_frequent = f1_score(y_true, y)
acc_most_frequent = accuracy_score(y_true, y)

results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

display(results_classification)

,model,task,f1,accuracy
0,Most Frequent class,task10,0.8641,0.760718


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [16]:
### Ваш код ###
from sklearn.impute import SimpleImputer

X_fix = X.copy()

# X_fix['workclass'][4] = np.nan
# X_fix['education'][0] = np.nan
# X_fix['education-num'][3] = pd.NA
# X_fix['fnlwgt'][4] = np.nan

if X_fix.isnull().sum().sum() > 0:
    # В данных есть пропуски.
    s_imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    X_fix.replace({pd.NA: np.nan}, inplace=True) # Заменить pd.NA на np.nan
    # Восстановить названия колонок и типы данных.
    X_fix = pd.DataFrame(s_imp.fit_transform(X_fix), columns=X.columns).astype(X.dtypes.to_dict())
else:
    # В данных нет пропусков.
    pass

display(X_fix.head(5))

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [17]:
### Ваш код ###
X_numerical_columns = X_fix.select_dtypes(include=[np.number]).columns.to_list()
display(X_numerical_columns)

X_categorical_columns = X_fix.select_dtypes(exclude=[np.number]).columns.to_list()
display(X_categorical_columns)

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [18]:
### Ваш код ###
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

numerical_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
])
transformer = ColumnTransformer(transformers=[
    ('numerical', numerical_transformer, X_numerical_columns),
    ('categorical', categorical_transformer, X_categorical_columns),
])

# X_fix = pd.DataFrame(transformer.fit_transform(X))
# print(X_fix.head(5))
# print(X_fix.shape)

lr_pipe = Pipeline([
    ('transformer', transformer),
    ('model', LogisticRegression()),
])
knn_pipe = Pipeline([
    ('transformer', transformer),
    ('model', KNeighborsClassifier()),
])
lin_pipe = Pipeline([
    ('transformer', transformer),
    ('model', LinearSVC()),
])

# display(sorted(sklearn.metrics.SCORERS.keys()))

f1_LR = cross_val_score(lr_pipe, X, y, scoring='f1').mean()
acc_LR = cross_val_score(lr_pipe, X, y, scoring='accuracy').mean()
f1_KNN = cross_val_score(knn_pipe, X, y, scoring='f1').mean()
acc_KNN = cross_val_score(knn_pipe, X, y, scoring='accuracy').mean()
f1_SVM = cross_val_score(lin_pipe, X, y, scoring='f1').mean()
acc_SVM = cross_val_score(lin_pipe, X, y, scoring='accuracy').mean()

results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

display(results_classification[1:4])

,model,task,f1,accuracy
1,LogisticRegression,task13,0.904969,0.851091
2,KNeighborsClassifier,task13,0.886993,0.824782
3,LinearSVC,task13,0.906322,0.852914


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [19]:
### Ваш код ###
X_fix = X.copy()
X_fix.replace({'?': np.nan}, inplace=True) # Замена значения '?' самыми частыми значениями.

if X_fix.isnull().sum().sum() > 0:
    simp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    X_fix = pd.DataFrame(simp.fit_transform(X_fix), columns=X.columns).astype(X.dtypes.to_dict())

f1_LR = cross_val_score(lr_pipe, X_fix, y, scoring='f1').mean()
acc_LR = cross_val_score(lr_pipe, X_fix, y, scoring='accuracy').mean()
f1_KNN = cross_val_score(knn_pipe, X_fix, y, scoring='f1').mean()
acc_KNN = cross_val_score(knn_pipe, X_fix, y, scoring='accuracy').mean()
f1_SVM = cross_val_score(lin_pipe, X_fix, y, scoring='f1').mean()
acc_SVM = cross_val_score(lin_pipe, X_fix, y, scoring='accuracy').mean()

results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

display(results_classification[1:7])

,model,task,f1,accuracy
1,LogisticRegression,task13,0.904969,0.851091
2,KNeighborsClassifier,task13,0.886993,0.824782
3,LinearSVC,task13,0.906322,0.852914
4,LogisticRegression_impute,task14,0.904753,0.850641
5,KNeighborsClassifier_impute,task14,0.887173,0.824987
6,LinearSVC_impute,task14,0.905142,0.850825


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [20]:
### Ваш код ###

# Удалить строки содержащие значения '?'.
X_fix = X.copy()
display(X_fix.shape)
X_fix.replace({'?': np.nan}, inplace=True)
rows_with_nan = [index for index, row in X_fix.iterrows() if row.isnull().any()]
X_fix.drop(index=rows_with_nan, inplace=True)
display(X_fix.shape)

y_fix = y.copy()
display(y_fix.shape)
y_fix.drop(index=rows_with_nan, inplace=True)
display(y_fix.shape)

f1_LR_del_missings = cross_val_score(lr_pipe, X_fix, y_fix, scoring='f1').mean()
acc_LR_del_missings = cross_val_score(lr_pipe, X_fix, y_fix, scoring='accuracy').mean()
f1_KNN_del_missings = cross_val_score(knn_pipe, X_fix, y_fix, scoring='f1').mean()
acc_KNN_del_missings = cross_val_score(knn_pipe, X_fix, y_fix, scoring='accuracy').mean()
f1_SVM_del_missings = cross_val_score(lin_pipe, X_fix, y_fix, scoring='f1').mean()
acc_SVM_del_missings = cross_val_score(lin_pipe, X_fix, y_fix, scoring='accuracy').mean()

results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

display(results_classification[4:10])

(48842, 14)

(45222, 14)

(48842,)

(45222,)

,model,task,f1,accuracy
4,LogisticRegression_impute,task14,0.904753,0.850641
5,KNeighborsClassifier_impute,task14,0.887173,0.824987
6,LinearSVC_impute,task14,0.905142,0.850825
7,LogisticRegression_delete_missings,task15,0.901149,0.846845
8,KNeighborsClassifier_delete_missings,task15,0.882951,0.820574
9,LinearSVC_delete_missings,task15,0.902403,0.848503


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [21]:
### Ваш код ###
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

X_fix = X.copy()
X_fix.replace({'?': np.nan}, inplace=True) # Замена значения '?' самыми частыми значениями.

if X_fix.isnull().sum().sum() > 0:
    simp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    X_fix = pd.DataFrame(simp.fit_transform(X_fix), columns=X.columns).astype(X.dtypes.to_dict())

rf_pipe = Pipeline([
    ('transformer', transformer),
    ('model', RandomForestClassifier()),
])
gb_pipe = Pipeline([
    ('transformer', transformer),
    ('model', GradientBoostingClassifier()),
])

f1_RF = cross_val_score(rf_pipe, X_fix, y, scoring='f1').mean()
acc_RF = cross_val_score(rf_pipe, X_fix, y, scoring='accuracy').mean()
f1_GB = cross_val_score(gb_pipe, X_fix, y, scoring='f1').mean()
acc_GB = cross_val_score(gb_pipe, X_fix, y, scoring='accuracy').mean()

results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

display(results_classification[10:12])

,model,task,f1,accuracy
10,RandomForestClassifier,task16,0.904951,0.851071
11,GradientBoostingClassifier,task16,0.915395,0.866386


17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [22]:
### Ваш код ###
from sklearn.preprocessing import LabelEncoder

X_fix = X.copy()
X_fix.replace({'?': np.nan}, inplace=True) # Замена значения '?' самыми частыми значениями.

if X_fix.isnull().sum().sum() > 0:
    simp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    X_fix = pd.DataFrame(simp.fit_transform(X_fix), columns=X.columns).astype(X.dtypes.to_dict())

estimator = Pipeline(steps=[
    ('transformer', transformer),
    ('model', LogisticRegression()),
])
parameters = {
    'transformer__categorical__imputer__strategy': ['mean', 'median', 'most_frequent', 'constant'],
    'transformer__categorical__encoder': [
        LabelEncoder(),
        OneHotEncoder(handle_unknown='ignore', sparse=False),
    ],
    'transformer__numerical__scaler': [
        StandardScaler(),
        MinMaxScaler(),
    ],
    'model': [
        LogisticRegression(),
        KNeighborsClassifier(),
        LinearSVC(),
        RandomForestClassifier(),
        GradientBoostingClassifier(),
    ],
}

X_train, X_test, y_train, y_test = train_test_split(X_fix, y, train_size=0.8, random_state=RANDOM_STATE)
print((X_train.shape, X_test.shape, y_train.shape, y_test.shape))
gs = GridSearchCV(estimator=estimator, param_grid=parameters)
# print(sorted(gs.get_params().keys()))
gs.fit(X_train, y_train)

best_params = gs.best_params_
print('Параметры лучшей модели:\n', best_params)
f1_best = f1_score(y_test, gs.predict(X_test))
acc_best = accuracy_score(y_test, gs.predict(X_test))

results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

display(results_classification[12:])

((39073, 14), (9769, 14), (39073,), (9769,))
Параметры лучшей модели:
 {'model': GradientBoostingClassifier(), 'transformer__categorical__encoder': OneHotEncoder(handle_unknown='ignore', sparse=False), 'transformer__categorical__imputer__strategy': 'most_frequent', 'transformer__numerical__scaler': StandardScaler()}


,model,task,f1,accuracy
12,Best_Model,task17,0.914541,0.864879


**Вывод:**

- Наилучшая модель **GradientBoostingClassifier**
- Наилучшая стратегия заполнение пропусков **most_frequent**
- Наилучший кодировщик категориальных признаков **OneHotEncoder**
- Наилучший метод масштабирования **StandardScaler**
- accuracy для модели с текущими параметрами **0.864879**
- f1_score для модели с текущими параметрами **0.914541**

In [23]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.864100,0.760718
1,LogisticRegression,task13,0.904969,0.851091
2,KNeighborsClassifier,task13,0.886993,0.824782
3,LinearSVC,task13,0.906322,0.852914
4,LogisticRegression_impute,task14,0.904753,0.850641
5,KNeighborsClassifier_impute,task14,0.887173,0.824987
6,LinearSVC_impute,task14,0.905142,0.850825
7,LogisticRegression_delete_missings,task15,0.901149,0.846845
8,KNeighborsClassifier_delete_missings,task15,0.882951,0.820574
9,LinearSVC_delete_missings,task15,0.902403,0.848503
